# Recommender Systems 2021/22

### Practice - FunkSVD and SVD++ implemented with Python and Cython

FunkSVD is one of the simplest and most known matrix factorization models for rating prediction. It was proposed by Simon Funk in a now famous post on his website (with a cow grazing around Auckland): https://sifter.org/~simon/journal/20061211.html

SVD++ is an extension of FunkSVD that learns also the biases: global, user and item.

In [1]:
import time
import numpy as np

In [2]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: Movielens10M
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10126, feature occurrences: 128384, density 1.19E-03
	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10106, feature occurrences: 106820, density 9.90E-04
	ICM name: ICM_year, Value range: 6.00E+00 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00




In [3]:
URM_train

<69878x10681 sparse matrix of type '<class 'numpy.float64'>'
	with 8000043 stored elements in Compressed Sparse Row format>

### What do we need for FunkSVD and SVD++?

* Loss function
* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience


In [4]:
n_users, n_items = URM_train.shape

#### The two methods are based on two latent factor matrices $ W \in R^{U \times E} , V \in R^{E \times I}$ with E the embedding size, and biases $ \mu \in R , b_u \in R^{U}, b_i \in R^{I}$

#### How to compute the predictions
FunkSVD: $ \hat{r}_{ui} = \sum_{j=0}^{E} W_{uj}H_{ji}$

SVD++: $ \hat{r}_{ui} = \mu + b_u + b_i + \sum_{j=0}^{E} W_{uj}H_{ji}$


#### The loss function we are interested in minimizing are

$L_{FunkSVD} = |R - WH|_F + \alpha|W|_2 + \beta|H|_2$

$L_{SVD++} = |R - (\mu + b_u + b_i + WH)|_F + \alpha|W|_2 + \beta|H|_2 + \gamma (\mu + b_u + b_i) $

Notice that in this case the loss is the Frobenius norm, not the 2 norm, hence we want to minimize the prediction error only for the existing ratings not the missing ones. In practice this means one samples only among the observed interactions.

While this approach works well for rating prediction, it does not for ranking. A model must be trained also on negative data if it expected to learn how to distinguish between positive and negative data. A good strategy is to randomly sample unobserved interactions during training assigning them a rating of 0.

#### Gradients

$\frac{\partial}{\partial W} L = -2(R - WH)H + 2\alpha W $

$\frac{\partial}{\partial H} L = -2(R - WH)W + 2\alpha H $

$\frac{\partial}{\partial \mu} L = -2(R - WH) + 2\gamma \mu $

$\frac{\partial}{\partial b_u} L = -2(R - WH) + 2\gamma b_u $

$\frac{\partial}{\partial b_i} L = -2(R - WH) + 2\gamma b_i $

#### The update is going to be (we can remove the coefficients)
$ W = W - \frac{\partial}{\partial W}$, or 

$ W = W + l((R - WH)H - \alpha W)$, with $l$ the learning rate

... and similarly for the other parameters: $H$, $\mu$, $b_u$ and $b_i$.

## Step 1: We create the dense latent factor matrices
### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [5]:
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [6]:
user_factors

array([[0.81782221, 0.32623483, 0.14184231, ..., 0.49757594, 0.36608581,
        0.56447353],
       [0.97263807, 0.12753114, 0.10881674, ..., 0.49452373, 0.31971186,
        0.45143693],
       [0.29415558, 0.18812572, 0.8754103 , ..., 0.09303427, 0.79470217,
        0.25591769],
       ...,
       [0.19603707, 0.19376484, 0.8821932 , ..., 0.43127781, 0.67879198,
        0.0986332 ],
       [0.50865935, 0.64839804, 0.7321411 , ..., 0.63986981, 0.81789353,
        0.24421367],
       [0.18517082, 0.66859662, 0.98310463, ..., 0.77754424, 0.7527969 ,
        0.69585431]])

In [7]:
item_factors

array([[0.34346435, 0.2274097 , 0.41063912, ..., 0.59227481, 0.10454079,
        0.00179158],
       [0.71472291, 0.68239324, 0.25680354, ..., 0.79687973, 0.86179236,
        0.76447388],
       [0.98953008, 0.78386193, 0.15352571, ..., 0.12199865, 0.94654505,
        0.86581823],
       ...,
       [0.2179097 , 0.71748106, 0.01822124, ..., 0.88652553, 0.75715021,
        0.82133859],
       [0.6848683 , 0.20372009, 0.03798786, ..., 0.02428757, 0.52423793,
        0.25527629],
       [0.686449  , 0.64856278, 0.05689972, ..., 0.22115793, 0.43541106,
        0.56553645]])

## Step 2: We sample an interaction and compute the prediction of the current FunkSVD model

In [8]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

5006967

In [9]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

(user_id, item_id, rating)

(43649, 2479, 1.0)

In [10]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

3.229260877924115

#### The first predicted rating is a random prediction, essentially

### Step 3: We compute the prediction error and update the latent factor matrices

In [11]:
prediction_error = rating - predicted_rating
prediction_error

-2.229260877924115

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values latent factor matrices

### Which latent factors we modify? All the factors of the item and user we used

In [12]:
# Copy original value to avoid messing up the updates
H_i = item_factors[item_id,:]
W_u = user_factors[user_id,:]

In [13]:
H_i

array([0.18565804, 0.93119643, 0.99308814, 0.86275186, 0.95212794,
       0.1236942 , 0.52131264, 0.14175339, 0.40714719, 0.75519078])

In [14]:
W_u

array([0.66744419, 0.44321036, 0.00922959, 0.51790834, 0.76529915,
       0.6326098 , 0.75277024, 0.94348194, 0.77057488, 0.78101555])

#### Apply the update rule

In [15]:
learning_rate = 1e-4
regularization = 1e-5

In [16]:
user_factors_update = prediction_error * H_i - regularization * W_u
user_factors_update

array([-0.41388688, -2.07588421, -2.21385262, -1.92330414, -2.12254922,
       -0.27575297, -1.16214941, -0.31601473, -0.90764501, -1.68352507])

In [17]:
item_factors_update = prediction_error * W_u - regularization * H_i
item_factors_update

array([-1.48790907, -0.98804082, -0.02058509, -1.15456144, -1.70606098,
       -1.41025352, -1.67812645, -2.10326879, -1.71781651, -1.74109496])

In [18]:
user_factors[user_id,:] += learning_rate * user_factors_update 
item_factors[item_id,:] += learning_rate * item_factors_update

### Let's check what the new prediction for the same user-item interaction would be

In [19]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

3.2272237878516385

### The value is higher than before, we are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### WARNING: Initialization must be done with random non-zero values ... otherwise

In [20]:
user_factors = np.zeros((n_users, num_factors))
item_factors = np.zeros((n_items, num_factors))

predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

print("Prediction is {:.2f}".format(predicted_rating))

prediction_error = rating - predicted_rating

print("Prediction error is {:.2f}".format(prediction_error))


Prediction is 0.00
Prediction error is 1.00


In [21]:
H_i = item_factors[item_id,:]
W_u = user_factors[user_id,:]

user_factors[user_id,:] += learning_rate * (prediction_error * H_i - regularization * W_u)
item_factors[item_id,:] += learning_rate * (prediction_error * W_u - regularization * H_i)

In [22]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

print("Prediction after the update is {:.2f}".format(predicted_rating))
print("Prediction error is {:.2f}".format(rating - predicted_rating))

Prediction after the update is 0.00
Prediction error is 1.00


### Since the matrices are multiplied, if we initialize one of them as zero, the updates will always be zero and the model will not be able to learn.

### Let's put all together in a training loop.

In [23]:
URM_train_coo = URM_train.tocoo()

num_factors = 10
learning_rate = 1e-4
regularization = 1e-5

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

loss = 0.0
start_time = time.time()

for sample_num in range(1000000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
        
    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error**2
    
    # Copy original value to avoid messing up the updates
    H_i = item_factors[item_id,:]
    W_u = user_factors[user_id,:]  
    
    user_factors_update = prediction_error * H_i - regularization * W_u
    item_factors_update = prediction_error * W_u - regularization * H_i
    
    user_factors[user_id,:] += learning_rate * user_factors_update 
    item_factors[item_id,:] += learning_rate * item_factors_update    
    
    # Print some stats
    if (sample_num +1)% 100000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

Iteration 100000 in 3.39 seconds, loss is 2.64. Samples per second 29534.75
Iteration 200000 in 6.81 seconds, loss is 2.62. Samples per second 29347.44
Iteration 300000 in 9.86 seconds, loss is 2.59. Samples per second 30417.08
Iteration 400000 in 13.17 seconds, loss is 2.56. Samples per second 30380.99
Iteration 500000 in 16.15 seconds, loss is 2.53. Samples per second 30967.11
Iteration 600000 in 19.51 seconds, loss is 2.51. Samples per second 30750.35
Iteration 700000 in 22.75 seconds, loss is 2.48. Samples per second 30767.58
Iteration 800000 in 26.11 seconds, loss is 2.46. Samples per second 30644.79
Iteration 900000 in 29.29 seconds, loss is 2.44. Samples per second 30724.14
Iteration 1000000 in 32.55 seconds, loss is 2.41. Samples per second 30725.01


### What do we see? The loss generally goes down but may oscillate a bit.
### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [24]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 2603.74 seconds, or 43.40 minutes


### This model is relatively quick

### Let's see what we can do with Cython
### First step, just compile it. We do not have the data at compile time, so we put the loop in a function

In [25]:
%load_ext Cython

In [26]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape

    num_factors = 10
    learning_rate = 1e-4
    regularization = 1e-5

    user_factors = np.random.random((n_users, num_factors))
    item_factors = np.random.random((n_items, num_factors))

    loss = 0.0
    start_time = time.time()

    for sample_num in range(1000000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Copy original value to avoid messing up the updates
        H_i = item_factors[item_id,:]
        W_u = user_factors[user_id,:]  

        user_factors_update = prediction_error * H_i - regularization * W_u
        item_factors_update = prediction_error * W_u - regularization * H_i

        user_factors[user_id,:] += learning_rate * user_factors_update 
        item_factors[item_id,:] += learning_rate * item_factors_update    

        # Print some stats
        if (sample_num +1)% 100000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

In [27]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 100000 in 2.80 seconds, loss is 2.62. Samples per second 35729.33
Iteration 200000 in 5.65 seconds, loss is 2.58. Samples per second 35412.93
Iteration 300000 in 8.46 seconds, loss is 2.55. Samples per second 35450.74
Iteration 400000 in 11.42 seconds, loss is 2.53. Samples per second 35031.09
Iteration 500000 in 14.21 seconds, loss is 2.50. Samples per second 35182.58
Iteration 600000 in 16.98 seconds, loss is 2.48. Samples per second 35336.41
Iteration 700000 in 19.87 seconds, loss is 2.45. Samples per second 35222.50
Iteration 800000 in 22.69 seconds, loss is 2.43. Samples per second 35263.04
Iteration 900000 in 25.48 seconds, loss is 2.41. Samples per second 35318.26
Iteration 1000000 in 28.43 seconds, loss is 2.39. Samples per second 35168.39


In [28]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 2274.77 seconds, or 37.91 minutes


### The compiler is just porting in C all operations that the python interpreter would have to perform, dynamic tiping included. Have a look at the html reports in the Cython_examples folder

### Now try to add some types: If you use a variable only as a C object, use primitive tipes

* cdef int namevar
* cdef double namevar
* cdef float namevar
* cdef double[:] singledimensionarray
* cdef double[:,:] bidimensionalmatrix

### Some operations are still done with sparse matrices, those cannot be correctly optimized because the compiler does not know how what is the type of the data.

### To address this, we create typed arrays in which we put the URM_train data
For example, this operation: user_id = URM_train_coo.row[sample_index]

Becomes:
cdef int user_id
cdef int[:] URM_train_coo_row = URM_train_coo.row
user_id = URM_train_coo_row[sample_index]

### We can also skip the creation of the items_in_user_profile array and replace the np.random call with the faster native C function rand()


### We now use types for all main variables


In [29]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz
    
    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = 1e-4
    cdef double regularization = 1e-5
    
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data
    
    cdef double[:,:] user_factors = np.random.random((n_users, num_factors))
    cdef double[:,:] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update
                
    cdef double loss = 0.0
    cdef long start_time = time.time()

    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = rand() % n_interactions

        user_id = URM_train_coo_row[sample_index]
        item_id = URM_train_coo_col[sample_index]
        rating = URM_train_coo_data[sample_index]

        # Compute prediction
        predicted_rating = 0.0
        
        for factor_index in range(num_factors):
            predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]
 
        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Copy original value to avoid messing up the updates
        for factor_index in range(num_factors):
            
            H_i = item_factors[item_id,factor_index]
            W_u = user_factors[user_id,factor_index]  

            user_factors_update = prediction_error * H_i - regularization * W_u
            item_factors_update = prediction_error * W_u - regularization * H_i

            user_factors[user_id,factor_index] += learning_rate * user_factors_update 
            item_factors[item_id,factor_index] += learning_rate * item_factors_update    

        # Print some stats
        if (sample_num +1)% 500000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

In [30]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 500000 in 0.26 seconds, loss is 1.95. Samples per second 1920257.52
Iteration 1000000 in 0.36 seconds, loss is 1.66. Samples per second 2806649.96
Iteration 1500000 in 0.46 seconds, loss is 1.51. Samples per second 3252549.38
Iteration 2000000 in 0.56 seconds, loss is 1.40. Samples per second 3583733.98
Iteration 2500000 in 0.66 seconds, loss is 1.33. Samples per second 3782231.25
Iteration 3000000 in 0.76 seconds, loss is 1.27. Samples per second 3963662.37
Iteration 3500000 in 0.86 seconds, loss is 1.22. Samples per second 4085038.53
Iteration 4000000 in 0.96 seconds, loss is 1.18. Samples per second 4185541.57
Iteration 4500000 in 1.06 seconds, loss is 1.15. Samples per second 4263104.84
Iteration 5000000 in 1.15 seconds, loss is 1.12. Samples per second 4330999.02
Iteration 5500000 in 1.26 seconds, loss is 1.09. Samples per second 4358844.88
Iteration 6000000 in 1.35 seconds, loss is 1.07. Samples per second 4435524.89
Iteration 6500000 in 1.46 seconds, loss is 1.05. Samp

In [31]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 17.49 seconds, or 0.29 minutes


### Works nicely, let's put an additional for loop to do multiple epochs

In [32]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def train_multiple_epochs(URM_train, learning_rate_input, regularization_input, n_epochs):
    
    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz
    
    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = learning_rate_input
    cdef double regularization = regularization_input
    
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data

    cdef double[:,:] user_factors = np.random.random((n_users, num_factors))
    cdef double[:,:] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update
                
    cdef double loss = 0.0
    cdef long start_time = time.time()
    
    for n_epoch in range(n_epochs):

        loss = 0.0
        start_time = time.time()

        for sample_num in range(URM_train.nnz):

            # Randomly pick sample
            sample_index = rand() % n_interactions

            user_id = URM_train_coo_row[sample_index]
            item_id = URM_train_coo_col[sample_index]
            rating = URM_train_coo_data[sample_index]

            # Compute prediction
            predicted_rating = 0.0

            for factor_index in range(num_factors):
                predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]

            # Compute prediction error, or gradient
            prediction_error = rating - predicted_rating
            loss += prediction_error**2

            # Copy original value to avoid messing up the updates
            for factor_index in range(num_factors):

                H_i = item_factors[item_id,factor_index]
                W_u = user_factors[user_id,factor_index]  

                user_factors_update = prediction_error * H_i - regularization * W_u
                item_factors_update = prediction_error * W_u - regularization * H_i

                user_factors[user_id,factor_index] += learning_rate * user_factors_update 
                item_factors[item_id,factor_index] += learning_rate * item_factors_update    
            
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
     
        print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch+1, time.time() - start_time, loss/sample_num, samples_per_second))

    return np.array(user_factors), np.array(item_factors), loss, samples_per_second    

In [33]:
n_items = URM_train.shape[1]
learning_rate = 1e-3
regularization = 1e-5
    
user_factors, item_factors, loss, samples_per_second =  train_multiple_epochs(URM_train, learning_rate, regularization, 10)

Epoch 1 complete in in 1.78 seconds, loss is 5.799E-01. Samples per second 4493457.75
Epoch 2 complete in in 2.50 seconds, loss is 3.020E-01. Samples per second 3200270.12
Epoch 3 complete in in 2.09 seconds, loss is 2.202E-01. Samples per second 3829945.92
Epoch 4 complete in in 1.74 seconds, loss is 1.884E-01. Samples per second 4590142.63
Epoch 5 complete in in 2.48 seconds, loss is 1.716E-01. Samples per second 3220907.72
Epoch 6 complete in in 2.20 seconds, loss is 1.611E-01. Samples per second 3636394.87
Epoch 7 complete in in 1.84 seconds, loss is 1.538E-01. Samples per second 4337674.22
Epoch 8 complete in in 2.40 seconds, loss is 1.483E-01. Samples per second 3329157.48
Epoch 9 complete in in 2.04 seconds, loss is 1.442E-01. Samples per second 3919484.42
Epoch 10 complete in in 1.78 seconds, loss is 1.407E-01. Samples per second 4484293.32


### From 20 minutes of training time to a few seconds...